In [38]:
from datetime import datetime, timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
api_key = 'CKHSNRX8KKJ3FWSB925LNPTSZ'

In [39]:
def getResponse(url):
    return requests.get(url)

def getHTML(response):
    return response.text

In [40]:
@dataclass
class Holiday:
    name: str
    date: str
    
    def __str__ (self):
        return (f'{self.name} ({self.date})')

In [41]:
@dataclass
class Weather:
    date: str
    conditions: str

In [54]:
def get_weather():
    current_time = datetime.now()
    current_date = (f'{current_time.year}-{current_time.month}-{current_time.day}')
    dt = datetime.strptime(current_date, '%Y-%m-%d')
    start_temp = dt - timedelta(days=dt.weekday())
    end_temp = start_temp + timedelta(days=6)
    start = start_temp.strftime('%Y-%m-%d')
    end = end_temp.strftime('%Y-%m-%d')

    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Brooklyn%2C%20NY/{start}/{end}?unitGroup=metric&elements=datetime%2Cconditions&include=days&key={api_key}&contentType=json'

    response = requests.get(url)
    response
    json = response.json()

    global weather_list
    global weather_info
    weather_list = []
    for i in range(0,7):
        date = json['days'][i]['datetime']
        weather_desc = json['days'][i]['conditions']
        weather_info = Weather(date, weather_desc)
        weather_list.append(weather_info)
    
    return print(weather_list)

In [55]:
get_weather()

[Weather(date='2022-08-22', conditions='Rain, Partially cloudy'), Weather(date='2022-08-23', conditions='Rain, Partially cloudy'), Weather(date='2022-08-24', conditions='Partially cloudy'), Weather(date='2022-08-25', conditions='Partially cloudy'), Weather(date='2022-08-26', conditions='Rain, Partially cloudy'), Weather(date='2022-08-27', conditions='Partially cloudy'), Weather(date='2022-08-28', conditions='Partially cloudy')]


In [44]:
christmas = Holiday('Christmas', '2022-12-25')
vday = Holiday('Valentine\'s Day', '2022-02-14')
bday = Holiday('Birthday', '2022-02-13')

holidays = []
holidays.append(christmas)
holidays.append(vday)
holidays.append(bday)
holidays

[Holiday(name='Christmas', date='2022-12-25'),
 Holiday(name="Valentine's Day", date='2022-02-14'),
 Holiday(name='Birthday', date='2022-02-13')]

In [62]:
@dataclass
class HolidayList:
    inner_holidays: list

    def add_holiday(self, new_holiday_obj):
        if new_holiday_obj in self.inner_holidays:
            return(f'That holiday already exists. Please try again.')
        else:
            self.inner_holidays.append(new_holiday_obj)
            return(f'\nSuccess!\n{new_holiday_obj.name} has been added to the list.')

    def search_holidays(self, search_obj):
        for holiday in self.inner_holidays:
            if search_obj.name == holiday.name and search_obj.date == holiday.date:
                return holiday
        else:
            return('Holiday not found')

    def delete_holiday(self):
        holiday_found = False
        while holiday_found == False: 
            holiday_name_search = input('\nName: ')
            holiday_date_search = input('\nDate: ')
            for holiday in self.inner_holidays:
                if holiday_name_search == holiday.name and holiday_date_search == holiday.date:
                    self.inner_holidays.remove(holiday)
                    return (f'{holiday_name_search} deleted from list.')
            else:
                print('Holiday not found')
                continue

    def read_json(self):
        f = open(R'C:\Users\Jason Ruckel\Documents\advanced_python\holiday-assignment\holidays.json')
        data = json.load(f)
        for i in range(0, len(data)):
            name = data[i]['name']
            date = data[i]['date']
            new_holiday_obj = Holiday(name, date)
            print(f'Adding holiday {new_holiday_obj.name}.')
            self.add_holiday(new_holiday_obj)

    def save_to_json(self):
        holiday_list = [holiday.__dict__ for holiday in self.inner_holidays]
        with open(R'C:\Users\Jason Ruckel\Documents\advanced_python\holiday-assignment\holidays.json', 'w') as file:
            json.dump(holiday_list, file, indent = 4)
        return('Holidays were successfully saved to your file.')  

    def check_if_saved(self):
        holiday_list = [holiday.__dict__ for holiday in self.inner_holidays]
        json_holiday_list = []
        json_holiday_dict = [holiday.__dict__ for holiday in json_holiday_list]
        f = open(R'C:\Users\Jason Ruckel\Documents\advanced_python\holiday-assignment\holidays.json')
        data = json.load(f)
        for i in range(0, len(data)):
            name = data[i]['name']
            date = data[i]['date']
            new_holiday_obj = Holiday(name, date)
            json_holiday_list.append(new_holiday_obj)
        if holiday_list == json_holiday_dict:
            return True
        if holiday_list != json_holiday_dict:
            return False

    def scrape_holidays(self):
        for i in range (2020, 2025):
            url = f'https://www.timeanddate.com/holidays/us/{i}?hol=43119487'
            print(f'Scraping holidays from {i}')

            resp = getResponse(url)
            html = getHTML(resp)

            soup = BeautifulSoup(html, 'html.parser')

            rows = soup.find_all('tr', attrs = {'showrow'})

            for row in rows:
                date = row.find('th').text
                def mdy_to_ymd(d):
                    return datetime.strptime(d, '%b %d').strftime(f'{i}-%m-%d')
                formatted_date = mdy_to_ymd(date)
                name = row.find('a').text
                holiday = Holiday(name, formatted_date)
                self.add_holiday(holiday)
    
    def get_len(self):
        return len(self.inner_holidays)

    def convert_date_to_week_number(self, year, week_number):
        holidays_in_week = []
        for holiday in self.inner_holidays:
            from datetime import datetime
            string = holiday.date
            date = datetime.strptime(string, '%Y-%m-%d')

            y = date.year
            m = date.month
            d = date.day

            import datetime
            get_week = lambda y, m, d : datetime.date(y, m, d).isocalendar().week

            week_of_holiday = get_week(y, m, d)
            
            if y == year and week_of_holiday == week_number:
                holidays_in_week.append(holiday)
                continue
            else:
                continue
        return holidays_in_week  

    def get_current_holidays(self, year, week_number):
        holidays_in_week = []
        for holiday in self.inner_holidays:
            from datetime import datetime
            string = holiday.date
            date = datetime.strptime(string, '%Y-%m-%d')

            y = date.year
            m = date.month
            d = date.day

            import datetime
            get_week = lambda y, m, d : datetime.date(y, m, d).isocalendar().week

            week_of_holiday = get_week(y, m, d)
            
            if y == year and week_of_holiday == week_number:
                holidays_in_week.append(holiday)
                continue
            else:
                continue
        print('\nThese are the holidays for this week:')
        for holiday in holidays_in_week:
            print(f'\n{holiday.name} ({holiday.date})')
        for holiday in holidays_in_week:
            if holiday.date == weather_info.date:
                print(f'\n{holiday.name} ({holiday.date}) - {weather_info.conditions}')
            else:
                print('didnt work')

In [46]:
library = HolidayList(holidays)

In [ ]:
for holiday in holidays_in_week and weather_info in weather_list:
    if holiday.date == weather_info.date:
        print(f'\n{holiday.name} ({holiday.date}) - {weather_info.conditions}')

In [63]:
library.get_current_holidays(2022, 34)

In [ ]:
library.get_current_holidays(2022, 34)

In [32]:
library.convert_date_to_week_number(2020, 6)

[Holiday(name='Rosa Parks Day', date='2020-02-04'),
 Holiday(name='National Girls and Women in Sports Day', date='2020-02-05'),
 Holiday(name='Ronald Reagan Day', date='2020-02-06'),
 Holiday(name='National Wear Red Day', date='2020-02-07')]

In [34]:
datetime.now().isocalendar().week

34

In [49]:
library.get_len()

1565

In [48]:
library.scrape_holidays()

Scraping holidays from 2020
Scraping holidays from 2021
Scraping holidays from 2022
Scraping holidays from 2023
Scraping holidays from 2024


In [ ]:
library.read_json()

In [ ]:
library.save_to_json()

In [ ]:
library

In [ ]:

from datetime import datetime

string = '2022-07-13'
date = datetime.strptime(string, '%Y-%m-%d')

y = date.year
m = date.month
d = date.day

import datetime
get_week = lambda y, m, d : datetime.date(y, m, d).isocalendar().week

get_week(y, m, d)

In [ ]:
#adding a holiday for menu item 1
checking_holiday_date = True
while checking_holiday_date == True:
    new_holiday_name = input('\nName: ')
    new_holiday_date = input('\nDate: ')  
    try:
        datetime.strptime(new_holiday_date, '%Y-%m-%d')
        new_holiday_obj = Holiday(new_holiday_name, new_holiday_date)
        library.add_holiday(new_holiday_obj)
        checking_holiday_date = False
    except ValueError:
        print('Please format your date \'YYYY-MM-DD\'')
        continue